In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Mediapipe**
+ Hand + Pose

In [2]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 36.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [3]:
import os
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe 모델 초기화
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
hands = mp_hands.Hands()
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

In [4]:
num = '06'

In [5]:
INPUT_VIDEO_FOLDER = '/content/drive/MyDrive/기컴비_텀프/data/original'  # 입력 동영상 폴더 경로
for filename in os.listdir(INPUT_VIDEO_FOLDER):
    if filename.endswith('.mp4') & filename.startswith(num):
        print(filename)

06_L_친구.mp4
06_D_붕대.mp4
06_L_배고프다.mp4
06_R_의사.mp4
06_R_친구.mp4
06_F_의사.mp4
06_L_가렵다.mp4
06_F_가렵다.mp4
06_R_붕대.mp4
06_F_친구.mp4
06_D_친구.mp4
06_D_의사.mp4
06_U_골절.mp4
06_U_가렵다.mp4
06_R_골절.mp4
06_L_붕대.mp4
06_D_가렵다.mp4
06_U_친구.mp4
06_U_배고프다.mp4
06_F_붕대.mp4
06_F_배고프다.mp4
06_F_골절.mp4
06_L_의사.mp4
06_D_골절.mp4
06_R_배고프다.mp4
06_R_가렵다.mp4
06_U_의사.mp4
06_U_붕대.mp4
06_D_배고프다.mp4
06_L_골절.mp4
06_L_다리.mp4
06_U_병원.mp4
06_U_구급차.mp4
06_F_쓰러지다.mp4
06_F_병원.mp4
06_U_쓰러지다.mp4
06_D_구급차.mp4
06_D_병원.mp4
06_F_구급차.mp4
06_L_쓰러지다.mp4
06_L_구급차.mp4
06_R_다리.mp4
06_R_쓰러지다.mp4
06_F_다리.mp4
06_L_병원.mp4
06_R_구급차.mp4
06_R_병원.mp4
06_D_다리.mp4
06_D_쓰러지다.mp4
06_U_다리.mp4


In [6]:
INPUT_VIDEO_FOLDER = '/content/drive/MyDrive/기컴비_텀프/data/original'  # 입력 동영상 폴더 경로
OUTPUT_VIDEO_FOLDER = '/content/drive/MyDrive/기컴비_텀프/data/mediapipe'  # 출력 동영상 폴더 경로

for filename in os.listdir(INPUT_VIDEO_FOLDER):
    if filename.endswith('.mp4') & filename.startswith(num):
        # parts = filename.split('_')
        # word_dict = { 'WORD0163': '의사','WORD0044': '붕대','WORD0387': '골절',
        #              'WORD1204': '친구','WORD1101': '가렵다','WORD1197': '배고프다',
        #               'WORD0149': '구급차','WORD0571': '쓰러지다',
        #               'WORD1496': '병원','WORD1599': '다리', 'WORD1501':'smaple'}
        # word_id = parts[2]
        # korean_word = word_dict.get(word_id)
        # new_filename = f"{parts[3][4:]}_{parts[4][0]}_{korean_word}.mp4"
        new_filename = 'M_'+filename

        print(filename,end=' ')
        input_video_path = os.path.join(INPUT_VIDEO_FOLDER, filename)
        output_video_path = os.path.join(OUTPUT_VIDEO_FOLDER, new_filename)

        # 동영상 파일을 읽기 위한 준비
        cap = cv2.VideoCapture(input_video_path)

        # 출력 동영상 설정
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # 검은색 배경 생성
            black_background = np.zeros((height, width, 3), dtype=np.uint8)

            # BGR 이미지를 RGB로 변환
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # 손 및 포즈 처리
            hands_results = hands.process(image)
            pose_results = pose.process(image)

            # RGB 이미지를 BGR로 변환하여 출력을 위해 준비하지 않고,
            # 검은 배경에서 직접 손과 포즈 랜드마크를 그림
            image.flags.writeable = True

            # 손 랜드마크 그리기
            if hands_results.multi_hand_landmarks:
                for hand_landmarks in hands_results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(black_background, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # 포즈 랜드마크 그리기
            if pose_results.pose_landmarks:
                mp_drawing.draw_landmarks(black_background, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # 결과를 출력 동영상에 쓰기
            out.write(black_background)

        # 자원 해제
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print(new_filename)


06_L_친구.mp4 

/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


M_06_L_친구.mp4
06_D_붕대.mp4 M_06_D_붕대.mp4
06_L_배고프다.mp4 M_06_L_배고프다.mp4
06_R_의사.mp4 M_06_R_의사.mp4
06_R_친구.mp4 M_06_R_친구.mp4
06_F_의사.mp4 M_06_F_의사.mp4
06_L_가렵다.mp4 M_06_L_가렵다.mp4
06_F_가렵다.mp4 M_06_F_가렵다.mp4
06_R_붕대.mp4 M_06_R_붕대.mp4
06_F_친구.mp4 M_06_F_친구.mp4
06_D_친구.mp4 M_06_D_친구.mp4
06_D_의사.mp4 M_06_D_의사.mp4
06_U_골절.mp4 M_06_U_골절.mp4
06_U_가렵다.mp4 M_06_U_가렵다.mp4
06_R_골절.mp4 M_06_R_골절.mp4
06_L_붕대.mp4 M_06_L_붕대.mp4
06_D_가렵다.mp4 M_06_D_가렵다.mp4
06_U_친구.mp4 M_06_U_친구.mp4
06_U_배고프다.mp4 M_06_U_배고프다.mp4
06_F_붕대.mp4 M_06_F_붕대.mp4
06_F_배고프다.mp4 M_06_F_배고프다.mp4
06_F_골절.mp4 M_06_F_골절.mp4
06_L_의사.mp4 M_06_L_의사.mp4
06_D_골절.mp4 M_06_D_골절.mp4
06_R_배고프다.mp4 M_06_R_배고프다.mp4
06_R_가렵다.mp4 M_06_R_가렵다.mp4
06_U_의사.mp4 M_06_U_의사.mp4
06_U_붕대.mp4 M_06_U_붕대.mp4
06_D_배고프다.mp4 M_06_D_배고프다.mp4
06_L_골절.mp4 M_06_L_골절.mp4
06_L_